In [1]:
import os
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import torchvision
from torchvision.io import read_image
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from torchvision import datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import numpy as np
from PIL import Image

In [39]:
lr = 0.0002
n_epochs = 30
batch_size = 64
device = torch.device('cuda' if (torch.cuda.is_available()) else 'cpu')
print(device)

cpu


In [47]:
# Training local  
labels_path_train = 'data/train/labels/labels.csv'
img_dir_train = 'data/train/images'

# Testing local
labels_path_test = 'data/test/ids.csv'
img_dir_test = 'data/test/images'

# Training HPC
# labels_path_train = '/groups/CS156b/data/student_labels/train2023.csv'
# img_dir_train = '/groups/CS156b/data/train'

# Testing HPC
# labels_path_test = '/groups/CS156b/data/student_labels/test_ids.csv'
# img_dir_test = '/groups/CS156b/data/test'

df_train = pd.read_csv(labels_path_train)[:-1]
display(df_train.head())

df_test = pd.read_csv(labels_path_test)[:-1]
display(df_test.head())

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Pneumonia,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,0,0,0,0,0,train/pid50512/study1/view1_frontal.jpg,Female,68,Frontal,AP,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,1,1,1,1,1,train/pid21580/study2/view1_frontal.jpg,Female,87,Frontal,AP,-1.0,NaN,0.0,1.0,NaN,0.0,NaN,1.0,NaN
2,2,2,2,2,2,train/pid21580/study1/view1_frontal.jpg,Female,83,Frontal,AP,-1.0,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN
3,3,3,3,3,3,train/pid21580/study1/view2_lateral.jpg,Female,83,Lateral,NaN,-1.0,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN
4,4,4,4,4,4,train/pid33839/study1/view1_frontal.jpg,Male,41,Frontal,AP,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Unnamed: 0,Unnamed: 0.1,Id,Path
0,0,0,18,test/pid56785/study1/view1_frontal.jpg
1,1,1,19,test/pid56785/study1/view2_lateral.jpg
2,2,2,44,test/pid57943/study1/view1_frontal.jpg
3,3,3,45,test/pid57943/study2/view1_frontal.jpg
4,4,4,57,test/pid54805/study1/view1_frontal.jpg


In [48]:
def parse_labels(df):
    df.fillna(0, inplace=True)
    return df

class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = parse_labels(pd.read_csv(annotations_file)[:-1])
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        row = self.img_labels.iloc[idx]

        img_path = row['Path'].split('/')
        img_path = '/'.join(img_path[1:])
        img_path = os.path.join(self.img_dir, img_path)

        # image = read_image(img_path)
        image = Image.open(img_path) # PIL image for applying transform for pre-trained ResNet model 
        image = Image.fromarray(np.stack((image,)*3, axis=-1)) # convert to RGB
        label = torch.tensor(list(row[-9:])).float() # extract label, the last 9 columns

        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)

        return image, label

In [55]:
# Transform for ResNet 
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

training_data = CustomImageDataset(labels_path_train, img_dir_train, transform=transform)
test_data = CustomImageDataset(labels_path_test, img_dir_test, transform=transform)

train_size = int(0.8 * len(training_data))
val_size = len(training_data) - train_size
training_data, val_data = torch.utils.data.random_split(training_data, [train_size, val_size])

train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [56]:
# sample model with dropout = 0.1
model = nn.Sequential(
    nn.Conv2d(3, 8, kernel_size=(3, 3)),
    nn.BatchNorm2d(num_features=8),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Dropout(p=0.1),

    nn.Conv2d(8, 8, kernel_size=(3, 3)),
    nn.BatchNorm2d(num_features=8),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Dropout(p=0.1),

    nn.Flatten(),
    nn.Linear(126*126*8, 64),
    nn.ReLU(),
    nn.Linear(64, 9),
    # PyTorch implementation of cross-entropy loss includes softmax layer
).to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0.5, 0.999))

In [58]:
# store metrics
training_loss_history = np.zeros(n_epochs)
validation_loss_history = np.zeros(n_epochs)

for epoch in range(n_epochs):
    print(f'Epoch {epoch+1}/{n_epochs}:')
    train_total = 0
    train_correct = 0
    # train
    model.train()
    for i, data in enumerate(train_dataloader):
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        # forward pass
        output = model(images)
        # calculate categorical cross entropy loss
        loss = criterion(output, labels)
        # backward pass
        loss.backward()
        optimizer.step()

        # track training loss
        training_loss_history[epoch] += loss.item()
    
    training_loss_history[epoch] /= len(train_dataloader)
    print(f'Training Loss: {training_loss_history[epoch]:0.4f}')

    # validate
    with torch.no_grad():
        model.eval()
        for i, data in enumerate(val_dataloader):
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            # forward pass
            output = model(images)
            # find loss
            loss = criterion(output, labels)
            validation_loss_history[epoch] += loss.item()
        validation_loss_history[epoch] /= len(val_dataloader)
    print(f'Validation loss: {validation_loss_history[epoch]:0.4f}')

Epoch 1/30:
Training Loss: 0.3257
Validation loss: 0.3965
Epoch 2/30:
Training Loss: 0.3252
Validation loss: 0.3965
Epoch 3/30:
Training Loss: 0.3258
Validation loss: 0.3966
Epoch 4/30:
Training Loss: 0.3255
Validation loss: 0.3966
Epoch 5/30:
Training Loss: 0.3252
Validation loss: 0.3967


In [ ]:
# get predictions on test set